# <center> AWS-foryou </center>
### <center> examples </center>
---

### Example 1
Running sklearndiabetes.py as the user's algorithm.

In [1]:
import boto3
import numpy as np
import os
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import time

os.chdir("..")

from awsforyou import aws_foryou
import examples.sklearn_diabetes as sdb

Using TensorFlow backend.


In [6]:
# os.chdir("..")
# os.getcwd()

'/mnt/c/Users/winsu/Documents/Seattle/U-of-Washington/Masters-in-Data-Science/2019_Spring/DATA_515/final-project/AWS-foryou'

In [2]:
# writing the csv files for x and y (data_loc and target_loc respectively)
# datasize is 3 times larger than the standard diabetes dataset
sdb.get_diabetes(3)

In [3]:
# assign the locations of csv files
data_loc = './examples/x_diabetes.csv'
target_loc = './examples/y_diabetes.csv'

In [4]:
# reference start time
start = time.time()
best_model = sdb.run_sklearn_diabetes(data_loc, target_loc)
finish = time.time()
runtime = finish - start
print("runtime is %f seconds" % runtime)
print("best model is \n %s" % best_model)

linear regression score = 0.359401
best hyperparameters estimate from grid search = 
 SVR(C=30, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False) 
score from using best hyperparameters = 0.526935
begining 6-components PCA decomposition
percentage of variance explained = 0.898523
repeat grid search with PCA-transformed data
best hyperparameters estimate from grid search = 
 SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='sigmoid', max_iter=-1, shrinking=True, tol=0.001, verbose=False) 
score from using best hyperparameters = 0.515038
runtime is 13.635998 seconds
best model is 
 SVR(C=30, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)


In [8]:
df = aws_foryou.aws_foryou("run_sklearn_diabetes(data_loc='examples/x_diabetes.csv', target_loc='examples/y_diabetes.csv')", 'examples.sklearn_diabetes')
df

linear regression score = -0.957795
best hyperparameters estimate from grid search = 
 SVR(C=50, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='sigmoid', max_iter=-1, shrinking=True, tol=0.001, verbose=False) 
score from using best hyperparameters = 0.068943
begining 6-components PCA decomposition
percentage of variance explained = 0.963393
repeat grid search with PCA-transformed data
best hyperparameters estimate from grid search = 
 SVR(C=20, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='sigmoid', max_iter=-1, shrinking=True, tol=0.001, verbose=False) 
score from using best hyperparameters = 0.036158
point 1, iteration 1 complete.
linear regression score = -0.957795
best hyperparameters estimate from grid search = 
 SVR(C=50, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='sigmoid', max_iter=-1, shrinking=True, tol=0.001, verbose=False) 
score from using best hyperparameters = 0.068943
begining 6-comp

Failed to fetch Price for instance m5a.12xlarge in region eu-north-1
Failed to fetch Price for instance m5a.12xlarge in region ap-south-1
Failed to fetch Price for instance m5a.12xlarge in region eu-west-3
Failed to fetch Price for instance m5a.12xlarge in region eu-west-2
Failed to fetch Price for instance m5a.12xlarge in region ap-northeast-2
Failed to fetch Price for instance m5a.12xlarge in region sa-east-1
Failed to fetch Price for instance m5a.12xlarge in region ca-central-1
Failed to fetch Price for instance m5a.12xlarge in region us-west-1
Failed to fetch Price for instance m5a.24xlarge in region eu-north-1
Failed to fetch Price for instance m5a.24xlarge in region ap-south-1
Failed to fetch Price for instance m5a.24xlarge in region eu-west-3
Failed to fetch Price for instance m5a.24xlarge in region eu-west-2
Failed to fetch Price for instance m5a.24xlarge in region ap-northeast-2
Failed to fetch Price for instance m5a.24xlarge in region sa-east-1
Failed to fetch Price for insta

,instance_type,runtime,estimated_time_aws,region,spot_price,on_demand_price,est_cost_spot_price,est_cost_on_demand_price
0,c5.18xlarge,12.297211,5.940830,eu-north-1,0.9828,3.2760,0.001622,0.005406
1,c5.18xlarge,12.297211,5.940830,ap-south-1,1.0432,3.0600,0.001722,0.005050
2,c5.18xlarge,12.297211,5.940830,eu-west-3,1.0908,3.6360,0.001800,0.006000
3,c5.18xlarge,12.297211,5.940830,eu-west-2,1.1311,3.6360,0.001867,0.006000
4,c5.18xlarge,12.297211,5.940830,eu-west-1,1.2367,3.4560,0.002041,0.005703
5,c5.18xlarge,12.297211,5.940830,ap-northeast-2,1.0788,3.4560,0.001780,0.005703
6,c5.18xlarge,12.297211,5.940830,ap-northeast-1,1.1976,3.8520,0.001976,0.006357
7,c5.18xlarge,12.297211,5.940830,sa-east-1,1.4685,4.7160,0.002423,0.007782
8,c5.18xlarge,12.297211,5.940830,ca-central-1,1.0408,3.3480,0.001718,0.005525
9,c5.18xlarge,12.297211,5.940830,ap-southeast-1,1.1212,3.5280,0.001850,0.005822
